# Ingest toy version of MNIST digit data from sklearn

### Determine run parameters

In [1]:
# ----------------- Parameters for interactive development --------------
P = {
    "pipeline.run_environment": "dev",
    "pipeline.data_lake_root": "/pipeline-outputs/data-lake",
    "run.retry_nr": "1",
}

In [2]:
# - During automated runs parameters will be injected in the below cell -

In [3]:
# Parameters
P = {
    "pipeline.data_lake_root": "/pipeline-outputs/data-lake",
    "pipeline.run_environment": "ci",
    "pipeline.pipeline_run_id": "2caa8dce-5051-4596-a5f0-de5241f6a484",
    "pipeline.github.repository": "matiasdahl/dev-mnist-digits-demo-pipeline",
    "pipeline.github.workflow": "Run automated tests, pipeline and deploy results to static reporting site",
    "pipeline.github.runner_name": "Hosted Agent",
    "pipeline.github.run_id": "3710884777",
    "pipeline.github.actor": "matiasdahl",
    "pipeline.github.job": "run-tests-pipeline-and-persist-pipeline-outputs",
    "pipeline.github.base_ref": "",
    "pipeline.github.head_ref": "",
    "pipeline.github.sha": "ace0d01011d5e914f1921450dd5a59c11367b38d",
    "pipeline.github.ref": "refs/heads/development",
    "pipeline.github.ref_type": "branch",
    "pipeline.github.ref_name": "development",
    "pipeline.github.event_name": "schedule",
    "task.notebook": "notebooks/ingest.py",
    "task.max_nr_retries": "15",
    "run.retry_nr": "10",
    "task.timeout_s": "10",
    "task.num_cpus": 1,
    "_opentelemetry_traceparent": "00-2c7c60ef416f61cec4fdb89ff37718a6-579e3d9b9f17076d-01",
}


In [4]:
# -----------------------------------------------------------------------

---

### Simulate different types of failures (for testing timeout and retry logic)

In [5]:
from pynb_dag_runner.tasks.task_opentelemetry_logging import PydarLogger

logger = PydarLogger(P)

2022-12-16 06:39:30,521	INFO worker.py:842 -- Connecting to existing Ray cluster at address: 172.17.0.2:6379


In [6]:
import time, random


def maybe_crash(retry_nr: int, run_environment: str):
    if retry_nr == 2 and run_environment == "ci":
        time.sleep(1e6)

    max_retry_nr: int = 3 if run_environment == "dev" else 10

    if retry_nr < max_retry_nr:
        if random.random() < 0.1:
            time.sleep(1e6)
        else:
            raise Exception("Simulated exception failure from ingestion step notebook!")


maybe_crash(
    retry_nr=int(P["run.retry_nr"]), run_environment=P["pipeline.run_environment"]
)

### Notebook code

In [7]:
from sklearn import datasets

#
from common.io import datalake_root, write_numpy

In [8]:
digits = datasets.load_digits()

X = digits["data"]
y = digits["target"]

In [9]:
logger.log_value("data_shape", list(X.shape))
logger.log_value("target_shape", list(y.shape))

X.shape, y.shape

 - Logging data_shape (json) : [1797, 64]
 - Logging target_shape (json) : [1797]


((1797, 64), (1797,))

In [10]:
write_numpy(datalake_root(P) / "raw" / "digits.numpy", X)
write_numpy(datalake_root(P) / "raw" / "labels.numpy", y)